# Capstone Project for week 4 & 5

## Define my problem.
### Problem statement: Clustering 1 star Michelin restaurants spread all over the world. To select and group the 1st star restaurants, I will use Foursquare appplication and Machine learning in Python extract data and create a data table about those information.


# Describe the data
## The data includes:
### Distrinct.
### The longitute and latitude.
### The kind of restaurant (category) such as sea food, pizza, or something else.
## To solve my problem:
### Firstly, I need to collect those mentioned data.
### Secondly, I generate a data table with those information.
### Next, I base on the region (distrinct) to classify restaurant. (K-means)
### Next, I use visualizing map.
### Finally, I will check the algorithm for the problem.

In [3]:
!pip install geopy
!pip install folium

     |████████████████████████████████| 94 kB 4.2 MB/s  eta 0:00:01


In [4]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print("Importing libraries")

Importing libraries


In [8]:
CLIENT_ID = 'AXZXCNUKWJBFKDB0NUCXSVIUGN0C4YD30VDG4WBSRHDWUOER' # your Foursquare ID
CLIENT_SECRET = 'JKHR3VJRV2WKNQHDAKEZ4B3ODTFOYDWEEIO3PAIR0JRG1W3Y' # your Foursquare Secret
VERSION = '20210203' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
ACCESS_TOKEN = 'NXOUF1TVZ21F4HTDESV5UQIZTJLJ4ITMIDYZ4WUZSB5RUCXB' # your FourSquare Access Token

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('ACCESS_TOKEN:' + ACCESS_TOKEN)

Your credentails:
CLIENT_ID: AXZXCNUKWJBFKDB0NUCXSVIUGN0C4YD30VDG4WBSRHDWUOER
CLIENT_SECRET:JKHR3VJRV2WKNQHDAKEZ4B3ODTFOYDWEEIO3PAIR0JRG1W3Y
ACCESS_TOKEN:NXOUF1TVZ21F4HTDESV5UQIZTJLJ4ITMIDYZ4WUZSB5RUCXB


In [9]:
search_query = 'Michelin'
radius = 500
print(search_query + ' .... OK!')

Danang .... OK!


In [10]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=AXZXCNUKWJBFKDB0NUCXSVIUGN0C4YD30VDG4WBSRHDWUOER&client_secret=JKHR3VJRV2WKNQHDAKEZ4B3ODTFOYDWEEIO3PAIR0JRG1W3Y&ll=16.407,108.13&oauth_token=NXOUF1TVZ21F4HTDESV5UQIZTJLJ4ITMIDYZ4WUZSB5RUCXB&v=20210203&query=Danang&radius=500&limit=100'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '601a3a46d775ce3689623158'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 0,
  'suggestedBounds': {'ne': {'lat': 16.411500004500006,
    'lng': 108.13468226685104},
   'sw': {'lat': 16.402499995499994, 'lng': 108.12531773314895}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

In [54]:
import pandas as pd
import requests
import io

url = "https://github.com/Masdevallia/data-analysis-pipeline/blob/master/input/one-star-michelin-restaurants.csv?raw=true" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url)

download = requests.get(url).content

df = pd.read_csv(url, index_col=0)

df.head()

,year,latitude,longitude,city,region,zipCode,cuisine,price,url
name,,,,,,,,,
Kilian Stuba,2019,47.348580,10.17114,Kleinwalsertal,Austria,87568,Creative,$$$$$,https://guide.michelin.com/at/en/vorarlberg/kl...
Pfefferschiff,2019,47.837870,13.07917,Hallwang,Austria,5300,Classic cuisine,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...
Esszimmer,2019,47.806850,13.03409,Salzburg,Austria,5020,Creative,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...
Carpe Diem,2019,47.800010,13.04006,Salzburg,Austria,5020,Market cuisine,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...
Edvard,2019,48.216503,16.36852,Wien,Austria,1010,Modern cuisine,$$$$,https://guide.michelin.com/at/en/vienna/wien/r...


In [172]:
column_name = ['name', 'zipCode', 'region', 'latitude', 'longitude']
#column_name = ['name'] 

# Create a dataframe of Michelin restaurants
ml_tab = pd.DataFrame(columns=column_name)
ml_tab

,name,zipCode,region,latitude,longitude


In [173]:
zc_l = list(df["zipCode"].values)
rg_l = list(df["region"].values)
lt_l = list(df["latitude"].values)
lo_l = list(df["longitude"].values)
#mx

In [174]:
for j,i in enumerate(df.index):
    zc = zc_l[j]
    rg = rg_l[j]
    lat = lt_l[j]
    lon = lo_l[j]
    ml_tab = ml_tab.append({'name': str(i),
                            'zipCode': zc,
                            'region': rg,
                            'latitude': lat,
                            'longitude': lon,
                           }, ignore_index=True)


In [177]:
ml_tab.head()

,name,zipCode,region,latitude,longitude
0,Kilian Stuba,87568,Austria,47.348580,10.17114
1,Pfefferschiff,5300,Austria,47.837870,13.07917
2,Esszimmer,5020,Austria,47.806850,13.03409
3,Carpe Diem,5020,Austria,47.800010,13.04006
4,Edvard,1010,Austria,48.216503,16.36852


In [179]:
# one hot encoding
ml_tab_onehot = pd.get_dummies(ml_tab[['name']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ml_tab_onehot['region'] = ml_tab['region'] 

# move neighborhood column to the first column
fixed_columns = [ml_tab_onehot.columns[-1]] + list(ml_tab_onehot.columns[:-1])
ml_tab_onehot = ml_tab_onehot[fixed_columns]

In [182]:
ml_tab_onehot.head()

,region,108,21212,28+,360º,8 1/2 Otto e Mezzo - Bombana,A. Wong,Adam's,Addison,Agern,...,Zero Complex,Zhejiang Heen,Zi Yat Heen,aend,atelier Amaro,formel B,jū-ni,logy,wilks,Épure
0,Austria,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Austria,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Austria,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Austria,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Austria,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [229]:
ml_tab_grouped = ml_tab_onehot.groupby('region').mean().reset_index()
ml_tab_grouped.head()

,region,108,21212,28+,360º,8 1/2 Otto e Mezzo - Bombana,A. Wong,Adam's,Addison,Agern,...,Zero Complex,Zhejiang Heen,Zi Yat Heen,aend,atelier Amaro,formel B,jū-ni,logy,wilks,Épure
0,Austria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.083333,0.0,0.0,0.000000,0.0,0.0,0.0
1,California,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014493,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.014493,0.0,0.0,0.0
2,Chicago,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
3,Croatia,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
4,Czech Republic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0


In [236]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

ml_tab_grouped_clustering = ml_tab_grouped.drop('region', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ml_tab_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 4, 3, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=int32)

In [189]:
!pip install folium

     |████████████████████████████████| 94 kB 3.0 MB/s eta 0:00:011


In [230]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['region']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
ml_tab_grouped_sorted = pd.DataFrame(columns=columns)
ml_tab_grouped_sorted['region'] = ml_tab_grouped['region']

for ind in np.arange(ml_tab_grouped.shape[0]):
    ml_tab_grouped_sorted.iloc[ind, 1:] = return_most_common_venues(ml_tab_grouped.iloc[ind, :], num_top_venues)

ml_tab_grouped_sorted.head()

,region,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Austria,Das Loft,Kilian Stuba,Edvard,SHIKI,Le Ciel by Toni Mörwald,Pfefferschiff,Walter Bauer,Pramerl & the Wolf,Tian,Esszimmer
1,California,Osteria Mozza,Kenzo,Shunji,Hayato,Kato,Keiko à Nob Hill,In Situ,Maum,Maude,Bistro Na's
2,Chicago,EL Ideas,Entente,Roister,Goosefoot,Spiaggia,Dusek's (Board & Beer),Boka,Band of Bohemia,North Pond,Elizabeth
3,Croatia,Draga di Lovrana,360º,Monte,Noel,Pelegrini,Épure,Hill Street Tai Hwa Pork Noodle,House,Ho Hung Kee,Hirohisa
4,Czech Republic,La Degustation Bohême Bourgeoise,Field,Épure,House,Ho Hung Kee,Hirohisa,Hinds Head,Hill Street Tai Hwa Pork Noodle,Hide,Heron & Grey


In [231]:
ml_tab_ = ml_tab

#cl_lb = {'Cluster Labels': kmeans.labels_}

ml_tab_grouped_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ml_tab_merged = ml_tab

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
ml_tab_merged = ml_tab_merged.join(ml_tab_grouped_sorted.set_index('region'), on='region')

ml_tab_merged.head() # check the last columns!

,name,zipCode,region,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Kilian Stuba,87568,Austria,47.348580,10.17114,2,Das Loft,Kilian Stuba,Edvard,SHIKI,Le Ciel by Toni Mörwald,Pfefferschiff,Walter Bauer,Pramerl & the Wolf,Tian,Esszimmer
1,Pfefferschiff,5300,Austria,47.837870,13.07917,2,Das Loft,Kilian Stuba,Edvard,SHIKI,Le Ciel by Toni Mörwald,Pfefferschiff,Walter Bauer,Pramerl & the Wolf,Tian,Esszimmer
2,Esszimmer,5020,Austria,47.806850,13.03409,2,Das Loft,Kilian Stuba,Edvard,SHIKI,Le Ciel by Toni Mörwald,Pfefferschiff,Walter Bauer,Pramerl & the Wolf,Tian,Esszimmer
3,Carpe Diem,5020,Austria,47.800010,13.04006,2,Das Loft,Kilian Stuba,Edvard,SHIKI,Le Ciel by Toni Mörwald,Pfefferschiff,Walter Bauer,Pramerl & the Wolf,Tian,Esszimmer
4,Edvard,1010,Austria,48.216503,16.36852,2,Das Loft,Kilian Stuba,Edvard,SHIKI,Le Ciel by Toni Mörwald,Pfefferschiff,Walter Bauer,Pramerl & the Wolf,Tian,Esszimmer


In [232]:
import folium
import numpy as np

import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[47.516231, 14.550072], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ml_tab['latitude'],ml_tab['longitude'], ml_tab['name'], kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [233]:
ml_tab_merged.loc[ml_tab_merged['Cluster Labels'] == 0, ml_tab_merged.columns[[1] + list(range(5, ml_tab_merged.shape[1]))]]

,zipCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
81,60618,0,EL Ideas,Entente,Roister,Goosefoot,Spiaggia,Dusek's (Board & Beer),Boka,Band of Bohemia,North Pond,Elizabeth
82,60625,0,EL Ideas,Entente,Roister,Goosefoot,Spiaggia,Dusek's (Board & Beer),Boka,Band of Bohemia,North Pond,Elizabeth
83,60625,0,EL Ideas,Entente,Roister,Goosefoot,Spiaggia,Dusek's (Board & Beer),Boka,Band of Bohemia,North Pond,Elizabeth
84,60618,0,EL Ideas,Entente,Roister,Goosefoot,Spiaggia,Dusek's (Board & Beer),Boka,Band of Bohemia,North Pond,Elizabeth
85,60640,0,EL Ideas,Entente,Roister,Goosefoot,Spiaggia,Dusek's (Board & Beer),Boka,Band of Bohemia,North Pond,Elizabeth
86,60662,0,EL Ideas,Entente,Roister,Goosefoot,Spiaggia,Dusek's (Board & Beer),Boka,Band of Bohemia,North Pond,Elizabeth
87,NaN,0,EL Ideas,Entente,Roister,Goosefoot,Spiaggia,Dusek's (Board & Beer),Boka,Band of Bohemia,North Pond,Elizabeth
88,60657,0,EL Ideas,Entente,Roister,Goosefoot,Spiaggia,Dusek's (Board & Beer),Boka,Band of Bohemia,North Pond,Elizabeth
89,60607,0,EL Ideas,Entente,Roister,Goosefoot,Spiaggia,Dusek's (Board & Beer),Boka,Band of Bohemia,North Pond,Elizabeth
90,60608,0,EL Ideas,Entente,Roister,Goosefoot,Spiaggia,Dusek's (Board & Beer),Boka,Band of Bohemia,North Pond,Elizabeth


In [234]:
ml_tab_merged.loc[ml_tab_merged['Cluster Labels'] == 1, ml_tab_merged.columns[[1] + list(range(5, ml_tab_merged.shape[1]))]]

,zipCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
259,00 085,1,atelier Amaro,Senses,Épure,Heron & Grey,House,Ho Hung Kee,Hirohisa,Hinds Head,Hill Street Tai Hwa Pork Noodle,Hide
260,00 507,1,atelier Amaro,Senses,Épure,Heron & Grey,House,Ho Hung Kee,Hirohisa,Hinds Head,Hill Street Tai Hwa Pork Noodle,Hide


In [235]:
ml_tab_merged.loc[ml_tab_merged['Cluster Labels'] == 2, ml_tab_merged.columns[[1] + list(range(5, ml_tab_merged.shape[1]))]]

,zipCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,87568,2,Das Loft,Kilian Stuba,Edvard,SHIKI,Le Ciel by Toni Mörwald,Pfefferschiff,Walter Bauer,Pramerl & the Wolf,Tian,Esszimmer
1,5300,2,Das Loft,Kilian Stuba,Edvard,SHIKI,Le Ciel by Toni Mörwald,Pfefferschiff,Walter Bauer,Pramerl & the Wolf,Tian,Esszimmer
2,5020,2,Das Loft,Kilian Stuba,Edvard,SHIKI,Le Ciel by Toni Mörwald,Pfefferschiff,Walter Bauer,Pramerl & the Wolf,Tian,Esszimmer
3,5020,2,Das Loft,Kilian Stuba,Edvard,SHIKI,Le Ciel by Toni Mörwald,Pfefferschiff,Walter Bauer,Pramerl & the Wolf,Tian,Esszimmer
4,1010,2,Das Loft,Kilian Stuba,Edvard,SHIKI,Le Ciel by Toni Mörwald,Pfefferschiff,Walter Bauer,Pramerl & the Wolf,Tian,Esszimmer
...,...,...,...,...,...,...,...,...,...,...,...,...
544,CT5 4BP,2,Lyle's,Quilon,Roganic,Rogan & Co,River Café,Ritz Restaurant,Restaurant Tristan,Restaurant Hywel Jones by Lucknam Park,Red Lion Freehouse,Eipic
545,TN27 8AH,2,Lyle's,Quilon,Roganic,Rogan & Co,River Café,Ritz Restaurant,Restaurant Tristan,Restaurant Hywel Jones by Lucknam Park,Red Lion Freehouse,Eipic
546,CT2 0DB,2,Lyle's,Quilon,Roganic,Rogan & Co,River Café,Ritz Restaurant,Restaurant Tristan,Restaurant Hywel Jones by Lucknam Park,Red Lion Freehouse,Eipic
547,JE2 4TQ,2,Lyle's,Quilon,Roganic,Rogan & Co,River Café,Ritz Restaurant,Restaurant Tristan,Restaurant Hywel Jones by Lucknam Park,Red Lion Freehouse,Eipic
